In [6]:
import  pandas as pd
import numpy as np

In [7]:
file10 = "csv-origins/GE2010_byCounty.csv"
ge10 = pd.read_csv(file10)

# Cleaning 2010 Results

In [8]:
k10, n10 = ge10.keys(), ge10.size

In [9]:
#check out the value counts for each column
valno10 = dict.fromkeys(k10)
for k in k10:
    valno10[k] = getattr(ge10,k).value_counts()

In [10]:
#these columns are not useful
dropvar10 = ["CanAffilCommit", "Election", "VoteFor", "PartyAbbrev","OfficeSequence"]

In [11]:
#so lets drop them
ge10.drop(columns=dropvar10,inplace=True)

In [12]:
nocandbool10 = list(ge10.CanLastName == "NO CANDIDATE")
nocandindx10 = ge10[nocandbool10].index
ge10.drop(index=nocandindx10,inplace=True)

In [69]:
ge10.loc[[" - YES" in x for x in ge10["CanLastName"]], "PartyName"] = "NON-PARTISAN"
ge10.loc[[" - NO" in x for x in ge10["CanLastName"]], "PartyName"] = "NON-PARTISAN"

In [14]:
for p in ge10.CanLastName:
    for l in p:
        if l != l.capitalize():
            capl = p.replace(l, l.capitalize())
            ge10.replace(p,capl,inplace=True)

In [16]:
nulloc10 = ge10.isnull().values
nansdf10 = ge10[nulloc10].copy()
nans = nansdf10.PartyName.values[0]
nancandlist10 = nansdf10[['CanFirstName','CanLastName','OfficeName']].drop_duplicates(subset="CanLastName")
nancandlist10 = nancandlist10.sort_values(by="CanLastName")
nancandlist10 = nancandlist10.values

In [ ]:
namelist10 = []
for l in nancandlist10:
    namelist10.append(" ".join(l))

In [ ]:
with open("nancandlist10.txt", 'w') as output:
    for row in namelist10:
        output.write(str(row) + '\n')

In [41]:
write_ins10 = list(nansdf10["CanLastName"].drop_duplicates().values)
ge10[[x in write_ins10 for x in ge10.CanLastName]] = ge10[[x in write_ins10 for x in ge10.CanLastName]].replace(nans,"WRITE IN")

In [46]:
candidates10 = ge10[["CandidateID","CanFirstName", "CanLastName","PartyName","OfficeName"]].copy()
candidates10.drop_duplicates(inplace=True,subset="CandidateID")
candidates10.set_index(candidates10.CandidateID,inplace=True)
candidates10.drop(columns="CandidateID",inplace=True)
candidates10.sort_values(by="CandidateID",inplace=True)
candidates10

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
15694,RICH,BRAUER,REPUBLICAN,100TH REPRESENTATIVE
15695,BOB,FLIDER,DEMOCRATIC,101ST REPRESENTATIVE
15697,MARK STEVEN,KIRK,REPUBLICAN,UNITED STATES SENATOR
15698,"ALEXANDER ""ALEXI""",GIANNOULIAS,DEMOCRATIC,UNITED STATES SENATOR
15699,LeALAN M.,JONES,GREEN,UNITED STATES SENATOR
...,...,...,...,...
16496,STEVE,FUNK,WRITE IN,18TH REPRESENTATIVE
16497,DARREN,RAICHART,WRITE IN,UNITED STATES SENATOR
16498,LOWELL M.,SEIDA,WRITE IN,UNITED STATES SENATOR


In [48]:
runningno10 = ge10.OfficeName.value_counts()
rndict10 = dict(runningno10)
rnlist10 = []
for o in ge10.OfficeName:
    rnlist10.append(rndict10[o])
NumberRunning10 = pd.Series(rnlist10,name="NumberRunning")

In [49]:
bg10 = ge10[["OfficeName","BallotGroup"]].copy()
bg10 = bg10.merge(NumberRunning10,left_index=True, right_index=True)
bg10.drop_duplicates(subset="OfficeName",inplace=True,ignore_index=True)

In [50]:
bg10count = bg10.groupby("BallotGroup")
bg10dict = dict.fromkeys(bg10.BallotGroup.drop_duplicates())

In [51]:
for bg in bg10.BallotGroup.drop_duplicates():
    nrsum = bg10count.get_group(bg).NumberRunning.sum()
    bg10dict[bg] = nrsum

In [52]:
bg10list = []
for bg in bg10.BallotGroup:
    bg10list.append(bg10dict[bg])
bg10ser = pd.Series(bg10list, index=bg10.index, name="BallotGrpCount")

In [53]:
bg10 = bg10.merge(bg10ser,left_index=True,right_index=True)

In [54]:
indx = pd.MultiIndex.from_frame(bg10[["BallotGroup","OfficeName"]])
bg10.set_index(indx,inplace=True)

In [55]:
bg10.sort_values(by=["BallotGrpCount","NumberRunning"], ascending=[False,False],inplace=True)

In [56]:
bg10.drop(columns=["BallotGroup","OfficeName","BallotGrpCount"],inplace=True)

In [57]:
ballotgroups10 = bg10.copy()

In [58]:
votes10 = ge10[["CandidateID","County","Votes"]].copy()
indx2 = pd.MultiIndex.from_frame(ge10[["CandidateID","County"]])
votes10.set_index(indx2,inplace=True)
votes10.drop(columns=["CandidateID","County"],inplace=True)
votes10.sort_values(by=['CandidateID','Votes'],ascending=[True,False],inplace=True)

In [59]:
candidates10.head()

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
15694,RICH,BRAUER,REPUBLICAN,100TH REPRESENTATIVE
15695,BOB,FLIDER,DEMOCRATIC,101ST REPRESENTATIVE
15697,MARK STEVEN,KIRK,REPUBLICAN,UNITED STATES SENATOR
15698,"ALEXANDER ""ALEXI""",GIANNOULIAS,DEMOCRATIC,UNITED STATES SENATOR
15699,LeALAN M.,JONES,GREEN,UNITED STATES SENATOR


In [60]:
ballotgroups10.head()

NumberRunning
BallotGroup OfficeName                                     
B           GOVERNOR AND LIEUTENANT GOVERNOR            777
            ATTORNEY GENERAL                            408
            COMPTROLLER                                 408
            TREASURER                                   408
            SECRETARY OF STATE                          306

In [61]:
votes10.head()

Votes
CandidateID County         
15694       SANGAMON  32318
            MENARD     4085
            LOGAN      4071
15695       MACON     12464
            MOULTRIE   1893

# Export tables to csvs


In [62]:
candidates10.to_csv("csv-clean/candidates10.csv",index_label="CandidateID")
ballotgroups10.to_csv("csv-clean/ballotgroups10.csv",index_label=["BallotGroup","OfficeName"])
votes10.to_csv("csv-clean/votes10.csv",index_label=["CandidateID","County"])

# Cleaning 2014 Results

In [63]:
file14 = "csv-origins/GE2014_byCounty.csv"
ge14 = pd.read_csv(file14)

In [64]:
#start the same as with 2010
k14, n14 = ge14.keys(), ge14.size

In [65]:
valno14 = dict.fromkeys(k14)
for k in k14:
    valno14[k] = getattr(ge14,k).value_counts()

In [66]:
#drop the same as in 2010, also new column ID
dropvar14 = dropvar10 + ['ID']

In [67]:
ge14.drop(columns=dropvar14,inplace=True)

In [92]:
ge14.loc[[" - YES" in x for x in ge14["CanLastName"]], "PartyName"] = "NON-PARTISAN"
ge14.loc[[" - NO" in x for x in ge14["CanLastName"]], "PartyName"] = "NON-PARTISAN"

In [113]:
ge14.loc[(ge14.BallotGroup == "1A").values,"CanFirstName"] = "AMENDMENT"
ge14.loc[(ge14.BallotGroup == "1A").values,"PartyName"] = "NON-PARTISAN"

In [120]:
ge14.loc[(ge14.BallotGroup == "2A").values,"CanFirstName"] = "PROPOSITION"
ge14.loc[(ge14.BallotGroup == "2A").values,"PartyName"] = "NON-PARTISAN"

In [139]:
nulloc14 = ge14.isnull().values
nansdf14 = ge14[nulloc14]
nancandlist14 = nansdf14[['CanFirstName','CanLastName','OfficeName']].drop_duplicates(subset="CanLastName")
nancandlist14 = nancandlist14.sort_values(by="OfficeName")
nancandlist14 = nancandlist14.values

In [140]:
namelist14 = []
for l in nancandlist14:
    namelist14.append(" ".join(l))

In [141]:
with open("nancandlist14.txt", 'w') as output:
    for row in namelist14:
        output.write(str(row) + '\n')

In [142]:
write_ins14 = list(nansdf14["CanLastName"].drop_duplicates().values)
ge14[[x in write_ins14 for x in ge14.CanLastName]] = ge14[[x in write_ins14 for x in ge14.CanLastName]].replace(nans,"WRITE IN")

## Instead of making one giant multiindex, I thought it made sense to split the dataset into three tables to take advantage of primary and foreign keys.
## Frame "ge14" will split into "candidates14", "ballotgroups14", and "votes14"
### Frame "ballotgroup14" contains a new column, "NumberRunning", counting how many candidates there are for each office.

In [143]:
candidates14 = ge14[["CandidateID","CanFirstName", "CanLastName","PartyName","OfficeName"]].copy()
candidates14.drop_duplicates(inplace=True,subset="CandidateID")
candidates14.set_index(candidates14.CandidateID,inplace=True)
candidates14.drop(columns="CandidateID",inplace=True)
candidates14.sort_values(by="CandidateID")

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
19443,CHAPIN,ROSE,REPUBLICAN,51ST SENATE
19444,CHRIS E.,REIF,REPUBLICAN,7TH CIRCUIT - MITCHELL VACANCY
19445,MIKE,ATTERBERRY,REPUBLICAN,8TH CIRCUIT - POPE VACANCY
19446,THOMAS B.,EWING,DEMOCRATIC,9TH CIRCUIT - DANNER VACANCY
19447,DAVE,CARLSON,REPUBLICAN,12TH CIRCUIT - 1ST SUBCIRCUIT - A VACANCY
...,...,...,...,...
20242,HILAIRE F.,SHIOURA,WRITE IN,UNITED STATES SENATOR
20243,PHIL,COLLINS,WRITE IN,9TH CONGRESS
20244,TOM,DEMAS,WRITE IN,78TH REPRESENTATIVE


In [145]:
runningno14 = ge14.OfficeName.value_counts()
rndict14 = dict(runningno14)
rnlist14 = []
for o in ge14.OfficeName:
    rnlist14.append(rndict14[o])
NumberRunning14 = pd.Series(rnlist14,name="NumberRunning")

In [146]:
bg14 = ge14[["OfficeName","BallotGroup"]].copy()
bg14 = bg14.merge(NumberRunning14,left_index=True, right_index=True)
bg14.drop_duplicates(subset="OfficeName",inplace=True,ignore_index=True)

In [148]:
bg14count = bg14.groupby("BallotGroup")
bg14dict = dict.fromkeys(bg14.BallotGroup.drop_duplicates())

for bg in bg14.BallotGroup.drop_duplicates():
    nrsum = bg14count.get_group(bg).NumberRunning.sum()
    bg14dict[bg] = nrsum

bg14list = []
for bg in bg14.BallotGroup:
    bg14list.append(bg14dict[bg])
bg14ser = pd.Series(bg14list, index=bg14.index, name="BallotGrpCount")

In [149]:
bg14 = bg14.merge(bg14ser,left_index=True,right_index=True)

In [150]:
indx = pd.MultiIndex.from_frame(bg14[["BallotGroup","OfficeName"]])
bg14.set_index(indx,inplace=True)

In [151]:
bg14.sort_values(by=["BallotGrpCount","NumberRunning"], ascending=[False,False],inplace=True)

In [152]:
bg14.drop(columns=["BallotGroup","OfficeName","BallotGrpCount"],inplace=True)

In [153]:
ballotgroups14 = bg14.copy()

In [154]:
votes14 = ge14[["CandidateID","County","Votes"]].copy()
indx2 = pd.MultiIndex.from_frame(ge14[["CandidateID","County"]])
votes14.set_index(indx2,inplace=True)
votes14.drop(columns=["CandidateID","County"],inplace=True)
votes14.sort_values(by=['CandidateID','Votes'],ascending=[True,False],inplace=True)

In [155]:
candidates14.head()

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
19443,CHAPIN,ROSE,REPUBLICAN,51ST SENATE
19444,CHRIS E.,REIF,REPUBLICAN,7TH CIRCUIT - MITCHELL VACANCY
19445,MIKE,ATTERBERRY,REPUBLICAN,8TH CIRCUIT - POPE VACANCY
19446,THOMAS B.,EWING,DEMOCRATIC,9TH CIRCUIT - DANNER VACANCY
19447,DAVE,CARLSON,REPUBLICAN,12TH CIRCUIT - 1ST SUBCIRCUIT - A VACANCY


In [156]:
ballotgroups14.head()

NumberRunning
BallotGroup OfficeName                                     
B           GOVERNOR AND LIEUTENANT GOVERNOR            761
            COMPTROLLER                                 403
            SECRETARY OF STATE                          334
            ATTORNEY GENERAL                            306
            TREASURER                                   306

In [157]:
votes14.head()

Votes
CandidateID County          
19443       MACON      16658
            CHAMPAIGN  15501
            SHELBY      7116
            PIATT       6443
            DOUGLAS     5623

# Export tables to csvs

In [158]:
candidates14.to_csv("csv-clean/candidates14.csv",index_label="CandidateID")
ballotgroups14.to_csv("csv-clean/ballotgroups14.csv",index_label=["BallotGroup","OfficeName"])
votes14.to_csv("csv-clean/votes14.csv",index_label=["CandidateID","County"])

# Will continue in the same way with GE18

In [ ]:
file18 = "csv-origins/GE2018_byCounty.csv"
ge18 = pd.read_csv(file18)

In [ ]:
k18, n18 = ge18.keys(), ge18.size

In [ ]:
valno18 = dict.fromkeys(k18)
for k in k18:
    valno18[k] = getattr(ge18,k).value_counts()

In [ ]:
dropvar18 = dropvar14

In [ ]:
ge18.drop(columns=dropvar18,inplace=True)

In [ ]:
candidates18 = ge18[["CandidateID","CanFirstName", "CanLastName","PartyName","OfficeName"]].copy()
candidates18.drop_duplicates(inplace=True,subset="CandidateID")
candidates18.set_index(candidates18.CandidateID,inplace=True)
candidates18.drop(columns="CandidateID",inplace=True)
candidates18.sort_values(by="CandidateID")

In [ ]:
nulloc18 = candidates18.PartyName.isnull().values
nansdf18 = candidates18[nulloc18]
nans = nansdf18.PartyName.values[0]

In [ ]:
candidates18.PartyName.replace(nans,"NON-PARTISAN",inplace=True)

In [ ]:
runningno18 = ge18.OfficeName.value_counts()
rndict18 = dict(runningno18)
rnlist18 = []
for o in ge18.OfficeName:
    rnlist18.append(rndict18[o])
NumberRunning18 = pd.Series(rnlist18,name="NumberRunning")

In [ ]:
bg18 = ge18[["OfficeName","BallotGroup"]].copy()
bg18 = bg18.merge(NumberRunning18,left_index=True, right_index=True)
bg18.drop_duplicates(subset="OfficeName",inplace=True,ignore_index=True)

In [ ]:
bg18count = bg18.groupby("BallotGroup")
bg18dict = dict.fromkeys(bg18.BallotGroup.drop_duplicates())

In [ ]:
for bg in bg18.BallotGroup.drop_duplicates():
    nrsum = bg18count.get_group(bg).NumberRunning.sum()
    bg18dict[bg] = nrsum

In [ ]:
bg18list = []
for bg in bg18.BallotGroup:
    bg18list.append(bg18dict[bg])
bg18ser = pd.Series(bg18list, index=bg18.index, name="BallotGrpCount")

In [ ]:
bg18 = bg18.merge(bg18ser,left_index=True,right_index=True)

In [ ]:
indx = pd.MultiIndex.from_frame(bg18[["BallotGroup","OfficeName"]])
bg18.set_index(indx,inplace=True)

In [ ]:
bg18.sort_values(by=["BallotGrpCount","NumberRunning"], ascending=[False,False],inplace=True)

In [ ]:
bg18.drop(columns=["BallotGroup","OfficeName","BallotGrpCount"],inplace=True)

In [ ]:
ballotgroups18 = bg18.copy()

In [ ]:
votes18 = ge18[["CandidateID","County","Votes"]].copy()
indx2 = pd.MultiIndex.from_frame(ge18[["CandidateID","County"]])
votes18.set_index(indx2,inplace=True)
votes18.drop(columns=["CandidateID","County"],inplace=True)
votes18.sort_values(by=['CandidateID','Votes'],ascending=[True,False],inplace=True)

In [ ]:
candidates18.head()

In [ ]:
ballotgroups18.head()

In [ ]:
votes18.head()


# Export tables to csvs


In [ ]:
candidates18.to_csv("csv-clean/candidates18.csv",index_label="CandidateID")
ballotgroups18.to_csv("csv-clean/ballotgroups18.csv",index_label=["BallotGroup","OfficeName"])
votes18.to_csv("csv-clean/votes18.csv",index_label=["CandidateID","County"])